#### Incorporating Final RAG components

In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
save_directory_faiss = "/content/drive/MyDrive/FAISS"

In [ ]:
# Using FAISS vector database - use for faster search
vector_db_faiss = FAISS.from_documents(documents = split_documents, embedding = embeddings)
vector_db_faiss.save_local(save_directory_faiss)

In [ ]:
vector_db_faiss2 = FAISS.load_local(save_directory_faiss, embeddings, allow_dangerous_deserialization=True)

In [ ]:
retriever_faiss = vector_db_faiss.as_retriever(search_kwargs={"k": 10})
retriever_faiss_mmr = vector_db_faiss.as_retriever(search_type = "mmr", fetch_k =50, search_kwargs={"k": 5, "lambda_mult": 0.5})
retriever_faiss_sst = vector_db_faiss.as_retriever(search_type = "similarity_score_threshold", search_kwargs={"k": 5, "score_threshold": 0.8})

In [ ]:
# persist_directory = "/content/drive/MyDrive/data/ChromaDB"
# # Using ChromaDB vector data base - Use if data base has a  very large size
# vector_db_chroma = Chroma.from_documents(documents=split_documents, embedding=embeddings, persist_directory=persist_directory,)
# vector_db_chromas = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
# retriever_chroma = vector_db_chroma.as_retriever(search_kwargs={"k": 5})
# retriever_chroma_mmr = vector_db_chroma.as_retriever(search_type = "mmr", search_kwargs={"k": 5, "lambda_mult": 0.5})
# retriever_chroma_sst = vector_db_chroma.as_retriever(search_type = "similarity_score_threshold", search_kwargs={"k": 5, "score_threshold": 0.8})

In [ ]:
# Retrival using MultiQuery
llm_retriever = ChatOpenAI(temperature=0)

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=retriever_faiss_mmr, llm=llm_retriever
)

In [ ]:
results = vector_db_faiss2.similarity_search_with_score("Shloka", k = 50)

In [ ]:
documents = []
scores = []
for doc, score in results:
    documents.append(doc)
    scores.append(score)

In [ ]:
documents[0].metadata["source"]

In [ ]:
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [ ]:
llm = ChatOpenAI(openai_api_key=openai_api_key, temperature=0.9, model='gpt-4o')

In [ ]:
# Defining various prompt templates for different tasks


#System Roles

prompt_compilation = PromptTemplate(
    input_variables=["data"],
    template=''' You will be given some data below, and a topic by the user. You have to properly and comprehensively compile the data to be relevant to the topic and explain it's concepts clearly from only the given data. Do not make up information.
     {data}  '''

)

prompt_story_generation = PromptTemplate(
    input_variables=["data"],
    template=''' You will be given some data below, and a topic by the user. You have to properly and comprehensively make a fun kids' story to explain the concepts relevant to the user's query in the datain simple terms so that kids can understand . Do not make up information.
     {data}  '''

)

prompt_poem_generation = PromptTemplate(
    input_variables=["data"],
    template=''' You will be given some data below, and a topic by the user. You have to properly and comprehensively make a fun kids' poem to explain the concepts relevant to the user's query in the data, in simple terms so that kids can understand. Do not make up information.
     {data}  '''

)

prompt_image_generation = PromptTemplate(
    input_variables=["data"],
    template=''' You will be given some story or poem below, and a topic by the user. Generate a detailed prompt, under 500 characters of length, to generate an image based on story/poem and the topic provided by the user.
     {data}  '''

)


#Human Roles

human_template_compilation = "{topic}"
human_template_story_generation = "{topic}"
human_template_poem_generation = "{topic}"
human_template_image_generation = "{topic}"

In [ ]:
system_message_prompt_1 = SystemMessagePromptTemplate(prompt=prompt_compilation)
human_message_prompt_1 = HumanMessagePromptTemplate.from_template(human_template_compilation)
chat_prompt_compilation = ChatPromptTemplate.from_messages([system_message_prompt_1, human_message_prompt_1])

In [ ]:
system_message_prompt_2 = SystemMessagePromptTemplate(prompt=prompt_story_generation)
human_message_prompt_2 = HumanMessagePromptTemplate.from_template(human_template_story_generation)
chat_prompt_story_generation = ChatPromptTemplate.from_messages([system_message_prompt_2, human_message_prompt_2])

In [ ]:
system_message_prompt_3 = SystemMessagePromptTemplate(prompt=prompt_poem_generation)
human_message_prompt_3 = HumanMessagePromptTemplate.from_template(human_template_poem_generation)
chat_prompt_poem_generation = ChatPromptTemplate.from_messages([system_message_prompt_3, human_message_prompt_3])

In [ ]:
system_message_prompt_4 = SystemMessagePromptTemplate(prompt=prompt_image_generation)
human_message_prompt_4 = HumanMessagePromptTemplate.from_template(human_template_image_generation)
chat_prompt_image_generation = ChatPromptTemplate.from_messages([system_message_prompt_4, human_message_prompt_4])

In [ ]:
llm_compressor = ChatOpenAI(temperature=0, model='gpt-4')

compressor = LLMChainExtractor.from_llm(llm_compressor)
# compression_retriever = ContextualCompressionRetriever(base_compressor=compressor,
#                                                        base_retriever=vectordb2.as_retriever(search_type="mmr",search_kwargs={"k":8, "lambda_mult": 0.8}))

In [ ]:
i = 1
processes = ["compilation", "story_generation", "poem_generation", "image_generation"]

In [ ]:

input_query = input("Your query: ")
for process in processes:
  if process == "compilation":
    print("Fetching relevant documents.... \n\n")
    unique_docs_multi_query = retriever_from_llm.get_relevant_documents(query=input_query)
    print("Displaying relevant document sources... \n\n")
    for i in range(len(unique_docs_multi_query)):
      print(unique_docs_multi_query[i].metadata)
    print("Compressing chunks for information density... \n\n")
    input_data = compressor.compress_documents(unique_docs_multi_query, query=input_query)
    if (len(input_data)<2):
      input_data = unique_docs_multi_query
    print("\n")
    print(input_data)
    print("\n")
    chat_prompt_with_values = chat_prompt_compilation.format_prompt(topic=input_query, data=input_data)
    messages_compilation = chat_prompt_with_values.to_messages()
    print("\n")
    print("Compiling your data... \n\n")
    response = llm(messages_compilation)
    print(response.content)
    messages_compilation.append(AIMessage(content=response.content))
    print("\n\n\n\n")

  elif process == "story_generation":
      print("generating story... \n")

  elif process == "poem_generation":
      print("hello")

  elif process == "image_generation":
      print("hello")


# else:
#   # appending logic
#   for process in processes:
#     if process == "compilation":
#         print("Fetching relevant documents.... \n\n")
#         unique_docs_multi_query = retriever_from_llm.get_relevant_documents(query=input_query)
#         print("Displaying relevant document sources... \n\n")
#         for i in range(len(unique_docs_multi_query)):
#           print(unique_docs_multi_query[i].metadata)
#         print("Compressing chunks for information density... \n\n")
#         input_data = compressor.compress_documents(unique_docs_multi_query, query=input_query)
#         if (len(input_data)<2):
#           input_data = unique_docs_multi_query
#         print(input_data)
#         system_prompt_with_values = system_message_prompt.format(data = input_data)
#         human_prompt_with_values = human_message_prompt.format(topic = input_query)
#         messages_compilation.append(SystemMessage(content = system_prompt_with_values.content))
#         messages_compilation.append(HumanMessage(content = human_prompt_with_values.content))
#         print("Compiling your response... \n\n")
#         response = llm(messages_compilation)
#         print(response.content)
#         messages_compilation.append(AIMessage(content=response.content))
#         print("\n\n\n\n")

#     elif process == "story_generation":
#         print("hello")
#     elif process == "poem_generation":
#         print("hello")
#     elif process == "image_generation":
#         print("hello")

In [ ]:
!python --version

In [ ]:
len(messages_compilation)

In [ ]:
messages_compilation

In [ ]:
messages_compilation[0].content

In [ ]:
list = []

In [ ]:
for i in range(len(messages_compilation)):
  list.append(messages_compilation[i].content)

In [ ]:
list

In [ ]:
# Define file paths
system_messages_file = './responses/system/system_messages.txt'
human_messages_file = './responses/human/human_messages.txt'
ai_messages_file = './responses/ai/ai_messages.txt'

# Initialize counters for each type of message
system_index = 1
human_index = 1
ai_index = 1

# Open files for writing
with open(system_messages_file, 'w') as sys_file, open(human_messages_file, 'w') as hum_file, open(ai_messages_file, 'w') as ai_file:
    # Iterate through the messages list
    for message in messages:
        # Check the type of the message and write to the corresponding file
        if isinstance(message, SystemMessage):
            sys_file.write(f"SystemMessage index {system_index}:\n{message.content}\n\n")
            system_index += 1
        elif isinstance(message, HumanMessage):
            hum_file.write(f"HumanMessage index {human_index}:\n{message.content}\n\n")
            human_index += 1
        elif isinstance(message, AIMessage):
            ai_file.write(f"AIMessage index {ai_index}:\n{message.content}\n\n")
            ai_index += 1

print("Messages have been written to their respective files.")

### **Code for various methods used for integration into RAG**

#### Base RAG

In [ ]:
llm = OpenAI(temperature=0.9, max_tokens=2048)

In [ ]:
vector_db = FAISS.from_documents(documents = split_documents, embedding = embeddings)

In [ ]:
retriever = vector_db.as_retriever(search_kwargs={"k": 10})
q = input("Please ask your query : ")
# should fetch  n relevant documents
rdocs = retriever.get_relevant_documents(q)

In [ ]:
print(type(rdocs))
print(len(rdocs))
rdocs

In [ ]:
#design prompt template
from langchain import PromptTemplate
prompt_template = """Use the following pieces of context to answer the question at the end, only using the data you are presented. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as detailed as possible. Always say "Namaskaram" at the end of the answer.
context = {context}
question = {question}
Answer: """
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [ ]:
# Modify the PromptTemplate to include context and question printing
class CustomPromptTemplate(PromptTemplate):
    def format_prompt(self, **kwargs):
        context = kwargs.get("context")
        question = kwargs.get("question")
        print(f"Question: {question}")
        print(f"Context: {context}")

        return super().format_prompt(**kwargs)

PROMPT = CustomPromptTemplate(template=prompt_template, input_variables=["context", "question"])

# Initialize the chain with the custom prompt template
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    input_key="query",
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)


In [ ]:
result = chain.invoke(q)

In [ ]:
print(result['result'])

In [ ]:
len(result['source_documents'])

#### Multi Query Advanced Retrieval

In [ ]:
!pip install chromadb

In [ ]:
vectordb2 = Chroma.from_documents(documents=split_documents, embedding=embeddings)

In [ ]:
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [ ]:
question = "What are arteries and what is their difference with the pulmonary artery"
llm = ChatOpenAI(temperature=0)

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb2.as_retriever(search_type="mmr",search_kwargs={"k":8}), llm=llm
)

In [ ]:
unique_docs = retriever_from_llm.get_relevant_documents(query=question)

In [ ]:
len(unique_docs)
unique_docs

In [ ]:
prompt_template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
text=PROMPT.format_prompt(
    context=unique_docs,
    question=question
)

In [ ]:
unique_docs

In [ ]:
llm.predict(text=PROMPT.format_prompt(
    context=unique_docs,
    question=question
).text)

In [ ]:
llm = ChatOpenAI(temperature=0, model='gpt-4')

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor,
                                                       base_retriever=vectordb2.as_retriever(search_type="mmr",search_kwargs={"k":8, "lambda_mult": 0.8}))

In [ ]:
compressed_docs = compression_retriever.get_relevant_documents(question)

In [ ]:
print(len(compressed_docs))
type(compressed_docs)

In [ ]:
for i in range(len(compressed_docs)):
  print(compressed_docs[i].page_content)

In [ ]:
output = llm.predict(text=PROMPT.format_prompt(
    context=compressed_docs,
    question=question
).text)

type(output)

In [ ]:
question = "What are arteries and what is their difference with the pulmonary artery"
llm = ChatOpenAI(temperature=0)

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb2.as_retriever(search_type="mmr",search_kwargs={"k":8}), llm=llm
)


#### Advanced RAG

In [ ]:
llm = ChatOpenAI(openai_api_key=openai_api_key, temperature=0, model='gpt-4o')

In [ ]:
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to Sankrit."),
    HumanMessage(content="What is the capital of India?"),
    AIMessage(content="New Delhi"),
    HumanMessage(content="How many states are there in India?")
]
llm(messages)

In [ ]:
prompt = PromptTemplate(
    input_variables=["data"],
    template=''' Give a funny poem, given the user's query and data given below
     {data}  '''
)

system_message_prompt = SystemMessagePromptTemplate(prompt=prompt)

In [ ]:
system_message_prompt.format(data = compressed_docs)

In [ ]:
human_template = "{query}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chat_prompt

In [ ]:
input_query = input("Please input your query: ")

In [ ]:
input_data = retriever_from_llm.get_relevant_documents(query=input_query)

In [ ]:
print(input_data)

In [ ]:
chat_prompt_with_values = chat_prompt.format_prompt(query=input_query, data=input_data)
messages = chat_prompt_with_values.to_messages()
messages

In [ ]:
llm = ChatOpenAI(temperature=0, model='gpt-4o')

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor,
                                                       base_retriever=vectordb2.as_retriever(search_type="mmr",search_kwargs={"k":8}))

In [ ]:
input_data = compression_retriever.get_relevant_documents(input_query)

In [ ]:
chat_prompt_with_values = chat_prompt.format_prompt(query=input_query, data=input_data)
messages = chat_prompt_with_values.to_messages()
messages

In [ ]:
response = llm(messages)
response

In [ ]:
print(response.content)

In [ ]:
chat = ChatOpenAI(openai_api_key=openai_api_key, streaming=True, callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]), verbose=True, temperature=0)
resp = chat(chat_prompt_with_values.to_messages())

##### Google Search Feature

In [ ]:
!pip install google-search-results

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
from google.colab import userdata
os.environ['SERPAPI_API_KEY']=userdata.get('SERP_API_KEY')

In [ ]:
llm = OpenAI(temperature=0)

In [ ]:
tool_names = ["serpapi"]
tools = load_tools(tool_names)

In [ ]:
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [ ]:
agent.run(input_query)

##### Wolfram Alpha Integration

In [ ]:
tool_names = ["wolfram-alpha"]
tools = load_tools(tool_names)

#### Ensemble Retrieval

In [ ]:
!pip install rank_bm25

In [ ]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

In [ ]:
vector_db_bm25 = BM25Retriever.from_documents(documents = split_documents, embedding = embeddings)
vector_db_bm25.k = 4

In [ ]:
vectordb_chroma = Chroma.from_documents(documents = split_documents, embedding = embeddings)
vectordb_chroma = vectordb_chroma.as_retriever(search_kwargs={"k": 4})

In [ ]:
ensemble_retriever = EnsembleRetriever(retrievers=[vector_db_bm25, vectordb_chroma], weights=[0.5, 0.5], verbose = True)

In [ ]:
ensemble_docs = ensemble_retriever.get_relevant_documents("what is the ovary?")
ensemble_docs